In [ ]:
# %cd  /home/superai2-4031/
# !git clone https://github.com/pytorch/fairseq.git
# %cd fairseq
# !pip install --editable ./
# !pip install subword-nmt
# !pip install torch==1.8.1+cu101 torchvision==0.9.1+cu101 \
# torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

In [20]:
from string import punctuation
import pandas as pd
path = "data/"
scb_mt_dataset = pd.read_csv('SCB_train_all_corpus.csv')
th = scb_mt_dataset['Thai']
en = scb_mt_dataset['English']

valid_en = open('ValidationSet/Valid1+2/dev3.en').read().translate(str.maketrans("","",punctuation.replace("'",""))).split("\n")
valid_th = open('ValidationSet/Valid1+2/dev3.th').read().translate(str.maketrans("","",punctuation.replace("'",""))).split("\n")

test_en = open('ValidationSet/Valid1+2/dev3.en').read().translate(str.maketrans("","",punctuation.replace("'",""))).split("\n")
test_th = open('ValidationSet/Valid1+2/dev3.th').read().translate(str.maketrans("","",punctuation.replace("'",""))).split("\n")

In [21]:
en.head(60)

0     accommodation and food service activities expa...
1       while food and construction materials increased
2                 banks' deposit continued to slow down
3                      world and us economic slowdown 2
4              chinese economic growth hit a 6-year-low
5     thus  farm income experienced consider-able gr...
6           however  export of other products increased
7            capital inflow in banking sector increased
8            the eurozone economy slightly decelerated
9           deposit and lending rate slightly increased
10    however  unemployment remained its concerning ...
11    meanwhile  agricultural and construction secto...
12                    economic growth target for 2004 6
13    meanwhile  the investment in construction dece...
14              ea solar nakornsawan co  ltd solar 9019
15    3 ) more pronounced recovery of private invest...
16        private investment increased at a slower pace
17             unemployment rate was low at 1.1 

In [22]:
th.tail(50)

279024    ขอโทษ   รอบ 3 ทุ่ม ครึ่ง ขาย หมด แล้ว   รอบ ใก...
279025    โอเค   เรา จอง ตั๋ว ไป ดู เดอะเลโก้ มูฟวี่ 2  ...
279026              ตั๋ว กับ ใบ เสร็จ จะ ส่ง ไป ใน โทรศัพท์
279027                         ด้ว ความ ยินดี   ดู ให้ สนุก
279028                         สวัสดี   มี อะไร ให้ ช่วย คะ
279029                สวัสดี   เรา อยาก ไป ดู หนัง บ่าย นี้
279030                     น่า สนุก นะ   อยาก ไป ดู อะไร คะ
279031            เรา ว่า เรา อยาก ไป ดู อิสนท์ อิทโรแมนติก
279032                             อยู่ ที่ ทูซอน แอริโซน่า
279033                                 โอเค   เดี๋ยว ดู ให้
279034    เซนจูรี่ เธียเตอร์ส ที่ ตลาดทูซอน มี ฉาย รอบ 1...
279035             รอบ บ่าย มี อะไร บ้าง   บ่าย อย่าง เดียว
279036                รอบ บ่าย มี ตอน 2 โมง 25 กับ 4 โมง 45
279037                            เรา รอบ 2 โมง 25 แล้ว กัน
279038    โอเค   ไป ดู อิสนท์ อิทโรแมนติก ที่ โรงเซนจูรี...
279039    ขอโทษ   รอบ 2 โมง 25 ขาย หมด แล้ว   อยาก ไป ดู...
279040    เยี่ยม   เดี๋ยว จอง ให้   เรีย

In [23]:
sel_en, sel_th = [],[]
for e,t in zip(en,th):
  if (e.count(' ') >= 5):
    sel_en.append(e)
    sel_th.append(t.strip())

In [24]:
len(sel_en)

236713

In [25]:
train_en_100k = sel_en
test_en_100k = aun_en
val_en_100k = valid_en
train_th_100k = sel_th
test_th_100k = aun_th
val_th_100k = valid_th

In [26]:
open(path+"train_en_100k",'w').write("\n".join(train_en_100k))
open(path+"test_en_100k",'w').write("\n".join(test_en_100k))
open(path+"val_en_100k",'w').write("\n".join(val_en_100k))
open(path+"train_th_100k",'w').write("\n".join(train_th_100k))
open(path+"test_th_100k",'w').write("\n".join(test_th_100k))
open(path+"val_th_100k",'w').write("\n".join(val_th_100k))

225076

In [27]:
!mkdir /home/superai2-4031/data/bpe

!subword-nmt learn-bpe -s 150000 \
< /home/superai2-4031/data/train_en_100k > /home/superai2-4031/data/bpe/src.bpe

!subword-nmt learn-bpe -s 150000 \
< /home/superai2-4031/data/train_th_100k > /home/superai2-4031/data/bpe/trg.bpe

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
 51%|#################8                 | 76362/150000 [02:45<02:39, 460.93it/s]
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
 62%|#####################6             | 92929/150000 [05:21<03:17, 288.64it/s]


In [28]:
!subword-nmt apply-bpe --dropout 0.1 --seed 56 -c /home/superai2-4031/data/bpe/src.bpe \
< /home/superai2-4031/data/train_en_100k > /home/superai2-4031/data/bpe/train.bpe.src

!subword-nmt apply-bpe  --dropout 0.1 --seed 56 -c /home/superai2-4031/data/bpe/src.bpe \
< /home/superai2-4031/data/test_en_100k > /home/superai2-4031/data/bpe/test.bpe.src

!subword-nmt apply-bpe  --dropout 0.1 --seed 56 -c /home/superai2-4031/data/bpe/src.bpe \
< /home/superai2-4031/data/val_en_100k > /home/superai2-4031/data/bpe/val.bpe.src

!subword-nmt apply-bpe  --dropout 0.1 --seed 56 -c /home/superai2-4031/data/bpe/trg.bpe \
< /home/superai2-4031/data/train_th_100k > /home/superai2-4031/data/bpe/train.bpe.trg

!subword-nmt apply-bpe  --dropout 0.1 --seed 56 -c /home/superai2-4031/data/bpe/trg.bpe \
< /home/superai2-4031/data/test_th_100k > /home/superai2-4031/data/bpe/test.bpe.trg

!subword-nmt apply-bpe  --dropout 0.1 --seed 56 -c /home/superai2-4031/data/bpe/trg.bpe \
< /home/superai2-4031/data/val_th_100k > /home/superai2-4031/data/bpe/val.bpe.trg

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


In [29]:
!ls /home/superai2-4031/data/bpe/

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
src.bpe       test.bpe.trg   train.bpe.trg  val.bpe.src
test.bpe.src  train.bpe.src  trg.bpe	    val.bpe.trg


In [30]:
# %cd  /home/superai2-4031/
# !git clone https://github.com/pytorch/fairseq.git
# %cd fairseq
# !git clone https://github.com/cython/cython.git
# %cd cython
# !git checkout 756b795
# !pip install --editable .
# %cd ..
# !pip install --editable .
# !python setup.py build_ext -inplace

In [31]:
!wc -l /home/superai2-4031/data/bpe/*bpe*

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
    76363 /home/superai2-4031/data/bpe/src.bpe
     3017 /home/superai2-4031/data/bpe/test.bpe.src
     3017 /home/superai2-4031/data/bpe/test.bpe.trg
   236712 /home/superai2-4031/data/bpe/train.bpe.src
   236712 /home/superai2-4031/data/bpe/train.bpe.trg
    92930 /home/superai2-4031/data/bpe/trg.bpe
     3017 /home/superai2-4031/data/bpe/val.bpe.src
     3017 /home/superai2-4031/data/bpe/val.bpe.trg
   654785 total


In [32]:
%cd /home/superai2-4031/data/bpe

/home/superai2-4031/data/bpe


In [33]:
#!python /content/fairseq/fairseq_cli/preprocess.py \
!python 'fairseq/fairseq_cli/preprocess.py' \
--trainpref ./train.bpe -s src -t trg \
--testpref ./test.bpe \
--validpref ./val.bpe \
--destdir ./data_enth_bin/ \
--workers 8


2022-03-10 22:31:58 | INFO | fairseq_cli.preprocess | Namespace(no_progress_bar=False, log_interval=100, log_format=None, log_file=None, tensorboard_logdir=None, wandb_project=None, azureml_logging=False, seed=1, cpu=False, tpu=False, bf16=False, memory_efficient_bf16=False, fp16=False, memory_efficient_fp16=False, fp16_no_flatten_grads=False, fp16_init_scale=128, fp16_scale_window=None, fp16_scale_tolerance=0.0, on_cpu_convert_precision=False, min_loss_scale=0.0001, threshold_loss_scale=None, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, user_dir=None, empty_cache_freq=0, all_gather_list_size=16384, model_parallel_size=1, quantization_config_path=None, profile=False, reset_logging=False, suppress_crashes=False, use_plasma_view=False, plasma_path='/tmp/plasma', criterion='cross_entropy', tokenizer=None, bpe=None, optimizer=None, lr_scheduler='fixed', simul_type=None, scoring='bleu', task='translation', source_lang='src', target_lang='trg', trainpref='./trai

In [34]:
%cd /data/bpe
!python fairseq/fairseq_cli/train.py ./data_enth_bin \
--arch transformer --max-epoch 150 --adam-betas '(0.9, 0.98)' --decoder-layers 6 --decoder-layers 8 --encoder-embed-dim 512\
--share-decoder-input-output-embed --batch-size 4096\
--optimizer adam --clip-norm 0.0 --scoring sacrebleu \
--lr 0.001 --lr-scheduler inverse_sqrt --warmup-updates 4000 --warmup-init-lr 1e-07 \
--dropout 0.1 --weight-decay 0.0 \
--criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
--save-dir checkpoints/transformer \
--max-tokens 4096 \
--update-freq 5 --device-id 2  \
--fp16 \
--keep-last-epochs 15 --distributed-world-size 1 
    #--patience 15

/home/superai2-4031/data/bpe
2022-03-10 22:32:31 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': None, 'path':

In [25]:
%cd /data/bpe/
!fairseq-generate ./data_enth_bin \
--path checkpoints/transformer/checkpoint_best.pt \
--batch-size 128 --beam 4 --remove-bpe \
| grep -P "[D|T]-[0-9]+" > log.txt
# | grep -P "D-[0-9]+" > log.txt

/home/superai2-4031/data/bpe
2022-03-10 21:24:50 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': None, 'pa

In [26]:
REF = []
translated = []
for line in open("log.txt",'r',encoding='utf8').readlines():
  splited_line = line.split('\t')
  if 'T' in splited_line[0]:
    REF.append(splited_line[1].strip())
  else:
    translated.append(splited_line[2].strip())
  # print(splited_line)
  # break
open('REF.txt','w',encoding='utf8').write('\n'.join(REF))
open('translated.txt','w',encoding='utf8').write('\n'.join(translated))

92869

In [27]:
!pwd


/home/superai2-4031/data/bpe


In [28]:
!cat translated.txt | sacrebleu REF.txt

{
 "name": "BLEU",
 "score": 0.1,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.0.0",
 "verbose_score": "3.3/0.2/0.0/0.0 (BP = 1.000 ratio = 1.326 hyp_len = 20826 ref_len = 15701)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.0.0"
}


Testing

In [ ]:
from string import punctuation
#path = "drive/MyDrive/Fairseq_Tutorial/data/"
path = "/home/superai2-4031/data/"

en_test = open('/home/superai2-4031/Official_Dataset/Aun/validate_en.txt').read().translate(str.maketrans("","",punctuation.replace("'",""))).split("\n")
th_test = open('/home/superai2-4031/Official_Dataset/Aun/validate_th_deep.txt').read().translate(str.maketrans("","",punctuation.replace("'",""))).split("\n")

In [ ]:
!mkdir /home/superai2-4031/data/bpe/aun_test

In [ ]:
!subword-nmt apply-bpe -c /home/superai2-4031/data/bpe/trg.bpe \
< /home/superai2-4031/data/test_th_100k > /home/superai2-4031/data/bpe/test.bpe.trg

In [ ]:
%cd /home/superai2-4031/data/bpe/aun_test

In [ ]:

!subword-nmt apply-bpe -c /home/superai2-4031/data/bpe/src.bpe \
< /home/superai2-4031/Official_Dataset/Aun/validate_en.txt > /home/superai2-4031/data/bpe/aun_test/test.bpe.src

!subword-nmt apply-bpe -c /home/superai2-4031/data/bpe/trg.bpe \
< /home/superai2-4031/Official_Dataset/Aun/validate_th_deep.txt > /home/superai2-4031/data/bpe/aun_test/test.bpe.trg



In [1]:
%cd /home/superai2-4031/data/bpe/

/home/superai2-4031/data/bpe


In [3]:
#!python /content/fairseq/fairseq_cli/preprocess.py \
!python '/home/superai2-4031/fairseq/fairseq_cli/preprocess.py' \
--trainpref ./train.bpe  \
--testpref ./aun_test/test.bpe -s src -t trg\
--destdir ./aun_test/data_enth_bin_test/ \
--workers 32

2022-03-10 19:46:56 | INFO | fairseq_cli.preprocess | Namespace(no_progress_bar=False, log_interval=100, log_format=None, log_file=None, tensorboard_logdir=None, wandb_project=None, azureml_logging=False, seed=1, cpu=False, tpu=False, bf16=False, memory_efficient_bf16=False, fp16=False, memory_efficient_fp16=False, fp16_no_flatten_grads=False, fp16_init_scale=128, fp16_scale_window=None, fp16_scale_tolerance=0.0, on_cpu_convert_precision=False, min_loss_scale=0.0001, threshold_loss_scale=None, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, user_dir=None, empty_cache_freq=0, all_gather_list_size=16384, model_parallel_size=1, quantization_config_path=None, profile=False, reset_logging=False, suppress_crashes=False, use_plasma_view=False, plasma_path='/tmp/plasma', criterion='cross_entropy', tokenizer=None, bpe=None, optimizer=None, lr_scheduler='fixed', simul_type=None, scoring='bleu', task='translation', source_lang='src', target_lang='trg', trainpref='./trai

In [ ]:
!fairseq-generate ./aun_test/data_enth_bin_test \
--path /home/superai2-4031/data/bpe/checkpoints/checkpoint_best.pt \
--batch-size 128 --beam 5 --remove-bpe \
| grep -P "[D|T]-[0-9]+" > log_test.txt

In [ ]:
REF = []
translated = []
for line in open("log_test.txt",'r',encoding='utf8').readlines():
  splited_line = line.split('\t')
  if 'T' in splited_line[0]:
    REF.append(splited_line[1].strip())
  else:
    translated.append(splited_line[2].strip())
  # print(splited_line)
  # break
open('REF_test.txt','w',encoding='utf8').write('\n'.join(REF))
open('translated_test.txt','w',encoding='utf8').write('\n'.join(translated))

In [ ]:
!pwd

In [ ]:
!cat translated_test.txt | sacrebleu REF_test.txt